<a href="https://colab.research.google.com/github/ccaballeroh/MCPR-2021/blob/main/04Extraction_Most_Relevant_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from pathlib import Path
import sys

IN_COLAB = "google.colab" in sys.modules

In [2]:
import warnings
warnings.filterwarnings("ignore")
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive/')
    ROOT = Path(r"./drive/My Drive/Translator-Attribution")
    sys.path.insert(0,f"{ROOT}/")
else:
    from helper.analysis import ROOT

# Extraction of Most Relevant Features

On this notebook, we extract the most relevant features in the classification process for each translator. In order to do this, we can retrieve the learned weights from a linear classifier (e.g., Logistic Regression, although a Support Vector Machine using a linear *kernel* also have those properties as well as the Naïve Bayes classifier) and get the $n$ largest. The corresponding $n$ features would thus be the most relevant for each class. In case of a binary classifier, the $n$ largest weights would correspond to the *positive* class, whereas the $n$ most negative weights would correspond to the *negative* class.

Since scikit-learn trains $N$ binary classifiers when given an N-class multiclass problem, we can retrieve the $n$ largest weights&mdash;and their corresponding features&mdash;for each classifier. This notebook saves to disk the $n$ most relevant features for each translator in the corpora for each feature set for a logistic regression classifier. The results are saved as bar plots and also tabular ($\LaTeX$) in the `results\figs\most` and `results\tables` folders respectively.

In [3]:
from helper.features import convert_data, plot_most_relevant, train_extract_most_relevant, save_tables

These are the files to process. They are the entirety of the feature sets obtained using [01Processing](./01Processing.ipynb).

In [4]:
from helper.analysis import JSON_FOLDER

## Most Relevant Features



We use $\chi^2$ statistic to select the $k$ most distinctive features in each feature set to train a Logistic Regression classifier. Then we extract the $n$ most important feature for each class (i.e., translator).

In [5]:
most_relevant = {}
num_of_features = 25
n_most_relevant = 10
args = {"k":num_of_features, "n":n_most_relevant}

for author in ["Ibsen", "Quixote"]:
    features_files = [file for file in JSON_FOLDER.iterdir() if file.name.startswith(author)]
    for file in features_files:
        data = convert_data(file=file)
        data = {**data, **args}
        most_relevant[file.stem] = train_extract_most_relevant(**data) 

        for translator in data["encoder"].classes_:
            plot_most_relevant(data=most_relevant[file.stem], translator=translator, file=file)
            df = most_relevant[file.stem][translator]
            save_tables(df=df, translator=translator, file=file)

<Figure size 432x288 with 0 Axes>

## Results

As an example, we can compare bigrams among the parallel translations of Ibsen (i.e., *Ghosts*).

In [6]:
key = "_".join(
    ["Ibsen",
     "Ghosts",
     "2grams",
    ])

In [7]:
for translator in most_relevant[key].keys():
    print(translator, ":\n", most_relevant[key][translator], 2*"\n")

Sharp :
         Feature    Weight
1       was the  0.707787
2        do n't  0.623912
3         as if  0.474847
4         it is  0.452557
5   manders but  0.445357
6         up to  0.444372
7   manders and  0.387682
8          i am  0.207958
9        at all  0.091415
10     going to  0.034201 


Archer :
        Feature    Weight
1         i 'm  0.790354
2       do not  0.729262
3        i 've  0.686770
4    PROPN why  0.583256
5   PROPN then  0.490356
6      can not  0.455052
7    not PROPN  0.451855
8     no doubt  0.407669
9     there 's  0.354415
10     you 're  0.272801 




Another example, from the other corpus. We show the 10 most distinctive cohesive markers sorrounded by their corresponding punctuation marks for each translator.

In [8]:
key = "_".join(
    ["Quixote",
     "cohesive",
     "punct"       
    ])
for translator in most_relevant[key].keys():
    print(translator, ":\n", most_relevant[key][translator], 2*"\n")

Jarvis :
         Feature    Weight
1        ; and,  1.783961
2         ; and  1.746763
3   . in short,  1.406864
4         . and  0.803417
5          also  0.593942
6       , since  0.516417
7         : and  0.460105
8   immediately  0.409644
9         " and  0.195828
10    therefore  0.171831 


Ormsby :
        Feature    Weight
1          and  2.397842
2   , however,  1.361017
3         "but  0.383121
4         "and  0.351197
5        " and  0.325242
6      , while  0.286503
7     although  0.138443
8       'there -0.001208
9   , although -0.016483
10        'but -0.265750 


Shelton :
        Feature    Weight
1         'and  1.711369
2        , yet  1.617163
3         'but  0.909684
4     likewise  0.904244
5   , although  0.799222
6       'there  0.633918
7       , and,  0.511501
8      , since  0.480780
9     although  0.477891
10   therefore  0.429038 


